In [1]:
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn import model_selection

In [2]:
df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
df.head(3)

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28


In [3]:
df['kfold'] = -1
df.head(3)

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,kfold
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,-1
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,-1
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,-1


In [5]:
df.shape

(9912, 15)

In [4]:
num_bins = int(np.floor(1 + np.log2(len(df))))
num_bins

14

In [6]:
# Use `cut` when you need to segment and sort data values into bins.
df.loc[:, "bins"] = pd.cut(df['Pawpularity'], bins = num_bins, labels=False)
df.head(3)

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,kfold,bins
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,-1,8
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,-1,5
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,-1,3


In [7]:
kfold = model_selection.StratifiedKFold(n_splits=10, # KFold = 10
                                        shuffle=True, 
                                        random_state=42)
kfold

StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

In [8]:
for fold, (t_, v_) in enumerate(kfold.split(X=df, y=df.bins.values)):
    df.loc[v_, 'kfold'] = fold
    
df.drop(columns=['bins'], inplace=True)
df.head(3)

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,kfold
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,1
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,5
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,1


In [9]:
df.tail(3)

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,kfold
9909,ffdf2e8673a1da6fb80342fa3b119a20,0,1,1,1,0,0,0,0,1,1,0,0,20,8
9910,fff19e2ce11718548fa1c5d039a5192a,0,1,1,1,0,0,0,0,1,0,0,0,20,2
9911,fff8e47c766799c9e12f3cb3d66ad228,0,1,1,1,0,0,0,0,0,0,0,0,30,4


In [10]:
set(df['kfold'])

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [11]:
def create_folds(df, num_splits):
    df['kfold'] = -1
    num_bins = int(np.floor(1 + np.log2(len(df))))

    df.loc[:, "bins"] = pd.cut(df['Pawpularity'], bins = num_bins, labels=False)

    kfold = model_selection.StratifiedKFold(n_splits=num_splits, # KFold
                                            shuffle=True, 
                                            random_state=42)
    
    for fold, (t_, v_) in enumerate(kfold.split(X=df, y=df.bins.values)):
        df.loc[v_, 'kfold'] = fold
    
    df.drop(columns=['bins'], inplace=True)

    return df

In [12]:
df_5 = create_folds(df, num_splits=5)
df_10 = create_folds(df, num_splits=10)
df_10.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,kfold
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,1
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,5
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,1
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,6
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,8
